In [11]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-explain-sentiment/resolve/main/explain-sentiment.json

In [34]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, T5Config
from malaya.torch_model.t5 import T5ForSequenceClassification

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [30]:
with open('explain-sentiment.json') as fopen:
    data = json.load(fopen)
    
texts, labels = [], []

labels_sentiment = ['negative', 'neutral', 'positive']
for d in data:
    try:
        l = json.loads(d[1])
        if not isinstance(l['sentiment'], str):
            continue
        if l['sentiment'] not in labels_sentiment:
            continue
        if not len(d[0]):
            continue
        texts.append(d[0])
        labels.append(labels_sentiment.index(l['sentiment']))
    except:
        pass
    
len(texts)

162895

In [43]:
list(zip(texts[:10], labels[:10]))

[('Terimakasih ya', 2),
 ('Enjin klik trus scroll ini yu', 1),
 ('Karya tangan anak bangsa sekarang patut diacungi jempol apalagi produknya yang yang bagus, banggalah kita memakai k',
  2),
 ('BABUN PUNYA AMZAR. TAK RETI HORMAT ORANG KEEEEEE???!!!!', 0),
 ('kau tak payah bagi data ni pukimak!! wahai pu', 0),
 ('Mau nikah ni biasanya', 1),
 ('Awak ada personal issue dengan Klang ke?', 1),
 ('Batu bersurat kuala berang tahun 1303.', 1),
 ('Dah tak tahu nak buat apa dengan info ni.', 0),
 ('hati hati ya semua. Kami .I anti main sentimen politik buka aib. Jual maruah manusia....I alert semua',
  0)]

In [31]:
np.unique(labels, return_counts = True)

(array([0, 1, 2]), array([63996, 66104, 32795]))

In [32]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(
    texts, labels, test_size = 0.1
)

In [36]:
config = T5Config.from_pretrained('mesolitica/nanot5-small-malaysian-cased')
config.num_labels = len(set(labels))
config.vocab = labels_sentiment

In [37]:
model = T5ForSequenceClassification.from_pretrained('mesolitica/nanot5-small-malaysian-cased', config = config)
_ = model.cuda()

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mesolitica/nanot5-small-malaysian-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-cased')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-4)

In [59]:
batch_size = 16
epoch = 100

best_dev_acc = -np.inf
patient = 3
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        loss, pred = model(**padded)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        losses.append(float(loss))
        
    dev_predicted = []
    for i in range(0, len(test_X), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        loss, pred = model(**padded)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [05:00<00:00, 30.45it/s]


epoch: 0, loss: 0.4947291410679668, dev_predicted: 0.6881133464180569


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [05:11<00:00, 29.43it/s]


epoch: 1, loss: 0.4765375747912435, dev_predicted: 0.6843719332679097


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [05:09<00:00, 29.64it/s]


epoch: 2, loss: 0.4514005017429948, dev_predicted: 0.6857826300294406


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [04:21<00:00, 35.05it/s]


epoch: 3, loss: 0.42360373258100387, dev_predicted: 0.6813052011776252


In [60]:
padded = tokenizer(texts[:10], padding = True, return_tensors = 'pt')
for k in padded.keys():
    padded[k] = padded[k].cuda()

In [61]:
model(**padded)[0].detach().cpu().numpy().argmax(axis = 1)

array([2, 1, 2, 0, 0, 1, 1, 1, 1, 0])

In [62]:
labels[:10]

[2, 1, 2, 0, 0, 1, 1, 1, 0, 0]

In [63]:
texts[:10]

['Terimakasih ya',
 'Enjin klik trus scroll ini yu',
 'Karya tangan anak bangsa sekarang patut diacungi jempol apalagi produknya yang yang bagus, banggalah kita memakai k',
 'BABUN PUNYA AMZAR. TAK RETI HORMAT ORANG KEEEEEE???!!!!',
 'kau tak payah bagi data ni pukimak!! wahai pu',
 'Mau nikah ni biasanya',
 'Awak ada personal issue dengan Klang ke?',
 'Batu bersurat kuala berang tahun 1303.',
 'Dah tak tahu nak buat apa dengan info ni.',
 'hati hati ya semua. Kami .I anti main sentimen politik buka aib. Jual maruah manusia....I alert semua']

In [64]:
labels_sentiment

['negative', 'neutral', 'positive']

In [65]:
model_ = T5ForSequenceClassification.from_pretrained('small')
_ = model_.cuda()

In [66]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()

    loss, pred = model(**padded)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1019/1019 [00:08<00:00, 119.27it/s]


In [67]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y, target_names = labels_sentiment,
        digits = 5
    )
)

              precision    recall  f1-score   support

    negative    0.71272   0.73407   0.72323      6276
     neutral    0.66692   0.66672   0.66682      6628
    positive    0.64844   0.61282   0.63012      3386

    accuracy                        0.68146     16290
   macro avg    0.67602   0.67120   0.67339     16290
weighted avg    0.68072   0.68146   0.68093     16290



In [68]:
model_.push_to_hub('mesolitica/sentiment-analysis-nanot5-small-malaysian-cased', safe_serialization = True)

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/sentiment-analysis-nanot5-small-malaysian-cased/commit/d083efe7f8cd67a1ed82b41c58a9d5784a9f2f91', commit_message='Upload T5ForSequenceClassification', commit_description='', oid='d083efe7f8cd67a1ed82b41c58a9d5784a9f2f91', pr_url=None, pr_revision=None, pr_num=None)

In [69]:
tokenizer.push_to_hub('mesolitica/sentiment-analysis-nanot5-small-malaysian-cased', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/sentiment-analysis-nanot5-small-malaysian-cased/commit/83c27565b162b115d7deae8758032101fe4e3dde', commit_message='Upload tokenizer', commit_description='', oid='83c27565b162b115d7deae8758032101fe4e3dde', pr_url=None, pr_revision=None, pr_num=None)